In [1]:
!pip install --upgrade pip
#!pip3 install tifffile

In [2]:
#to be run if not installed yet
!pip3 install pandas
!pip3 install pathlib
!pip3 install openpyxl
!pip3 install xlrd
!pip3 install xlwt
!pip3 install pyxlsb
!pip3 install hyperspy

In [3]:
import json
import os
import glob
import logging
from jsonschema import validate
import urllib.request
from pprint import pprint

In [6]:
#view all the metadata stored in the ZEISS TIFF file
import zeiss_tiff_meta.zeisstiffmeta as zm

In [7]:
src = 'FeMoOx_AntiA_04_1k5x_CN.tif'

# extract the full metadata as defined by Zeiss:
tab = zm.zeiss_meta(src)

In [8]:
#print(tab)
#first element in the tuple is a name used by Zeiss, second one is the more human understandable form, third is the value and the fouth is the unit, if it has a unit
#tab

In [9]:
#tab

In [10]:
class Imagedata:
    def __init__(self,tab):
        del tab[0]
        resultDictionary = {**dict((x1+"_value",x2) for x0, x1, x2, x3 in tab) , **dict((x1+"_unit",x3) for x0, x1, x2, x3 in tab)}
        resultDictionary_filtered={k: v for k, v in resultDictionary.items() if v != ''}
        self.resultDictionary_filtered_sorted=dict(sorted(resultDictionary_filtered.items()))

In [11]:
#pprint(Imagedata.__dict__)

In [12]:
metadata = Imagedata(tab)

In [13]:
#resultDictionary = dict((x1, {"value":x2,"unit":x3}) for x0, x1, x2, x3 in tab)

In [14]:
#resultDictionary_filtered_sorted

In [15]:
#testing Nicolas' metadataschemas reader
from metadataSchemaReader import MetadataSchemaReader

In [16]:
mySchema = "/Users/reetuelzajoseph/bwSyncShare/auto_data_extraction/SEM_schema.json"
with open(mySchema, 'r') as f:
    jsonSchema= json.load(f)
#print(type(jsonSchema["$defs"]),jsonSchema["$defs"])
readSchema=MetadataSchemaReader(jsonSchema)
#something wrong in conversion of the SEM json schema into a dictionary
#Q: how does the MetadataSchemaReader handle units?

FileNotFoundError: [Errno 2] No such file or directory: '/Users/reetuelzajoseph/bwSyncShare/auto_data_extraction/SEM_schema.json'

In [ ]:
 schema = readSchema.searchedSchema

In [ ]:
#pprint(schema)

In [ ]:
from attributeMapping import AttributeMapping
with open("mapSEM.json", 'r') as f:
    mapSEM_test = json.load(f)

In [ ]:
pprint(mapSEM_test)

In [ ]:
#type(resultDictionary_filtered_sorted)

In [ ]:
metadataDict=metadata.resultDictionary_filtered_sorted

In [ ]:
#pprint(metadataDict)

In [ ]:
for i in metadataDict:
    SEMmap = AttributeMapping(metadataDict,mapSEM_test,"mappedTerms")

In [ ]:
pprint(SEMmap.__dict__)

In [ ]:
pprint(SEMmap.__dict__.items())

In [ ]:
from mapSchema import MapSchema
filledSchema = MapSchema()
filledSchema = filledSchema.fillObject(schema, list(schema.keys()), None, SEMmap)
#pprint(filledSchema)

In [ ]:
pprint(filledSchema)

In [ ]:
with open("mappedMetadata.json", 'w') as f:
            json.dump(filledSchema, f)